# Projeto 1 - Ciência dos Dados

Nome: Matheus Barros

Nome: Giulia Sampaio

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import math

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'mcdonalds.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo mcdonalds.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Categorias
0  por isso eu prefiro o @mcdonalds_br https://t....           3
1  queria comer uma besteirinha, tipo um big tast...           3
2  @sergiosepsep @mcdonalds_br @ifood eu já tive ...           2
3                 @mvmarizinha @mcdonalds_br mentira           0
4  fui no @mcdonalds_br do @rapososhopping e fiz ...           4

In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Categorias
0  @mcdonalds_br tenho que me sujeitar a essas co...           4
1                      pq o @mcdonalds_br  e tão bom           3
2            @mcdonalds_br cadê o lanche do travis ?           2
3  @laiz_izabeli @mcdonalds_br da o combo todo po...           3
4  @jonitd_ @mcdonalds_br isso era pra ser um che...           3

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

**Analisar a opinião dos usuários a respeito da rede de fastfood mcdonald's, levando em conta as experiências do usuário.**

* (0) Muito Irrelevante - Um tweet que não reflete nada a respeito da opinião do usuário em relação ao mcdonalds.
 
* (1) Irrelevante - Um tweet que felita a opinião do usuário, mas pouco.

* (2) Neutro - Um tweet que não tenha defeitos nem qualidades, ou estão em equilíbrio.
 
* (3) Relevante - Tem uma opinião, boa ou ruim.

* (4) Muito Relevante - Tem uma opinião que reflita muito a imagem do Mcdonalds, boa ou ruim.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
train.Categorias = train.Categorias.astype('category')
test.Categorias = test.Categorias.astype('category')

train.Categorias.cat.categories = ['Muito Irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito Relevante']
test.Categorias.cat.categories = ['Muito Irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito Relevante']

train.head()

Treinamento         Categorias
0  por isso eu prefiro o @mcdonalds_br https://t....          Relevante
1  queria comer uma besteirinha, tipo um big tast...          Relevante
2  @sergiosepsep @mcdonalds_br @ifood eu já tive ...             Neutro
3                 @mvmarizinha @mcdonalds_br mentira  Muito Irrelevante
4  fui no @mcdonalds_br do @rapososhopping e fiz ...    Muito Relevante

In [6]:
train.Categorias.value_counts()

Muito Irrelevante    373
Relevante            139
Muito Relevante       89
Neutro                89
Irrelevante           60
Name: Categorias, dtype: int64

In [7]:
train.Categorias.value_counts(True)

Muito Irrelevante    0.497333
Relevante            0.185333
Muito Relevante      0.118667
Neutro               0.118667
Irrelevante          0.080000
Name: Categorias, dtype: float64

In [8]:
def cleanup(text):

    """
    Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """

    #import string

    punctuation = '[!-.:?\;_$%,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    link_sub = re.sub(r'http[^\s]*', '', text_subbed)
    #@[^\s]*

    return link_sub

In [9]:
#Muito Irrelevante Separação da Lista
m_irrel = train['Categorias'] == "Muito Irrelevante"
muito_irrelevante = train.loc[m_irrel, ["Treinamento"]].copy()
muito_irrelevante.head()

Treinamento
3                 @mvmarizinha @mcdonalds_br mentira
5  rt @thia6o0: @bolsonarosp @mcdonalds_br @slpng...
6  rt @rap24horasblog: combo do travis scott no m...
7  @mvmarizinha @mcdonalds_br sério?? kkkkkkkkkkk...
9  olá, @mcdonalds_br o @luansantana está há proc...

In [10]:
#Irrelevante Separação da Lista
irrel = train['Categorias'] == "Irrelevante"
irrelevante = train.loc[irrel, ["Treinamento"]].copy()
irrelevante.head()

Treinamento
331  @mcdonalds_br oh mano seis não responderam mai...
332  olha nem o cbo de volta no cardápio do @mcdona...
333  @cullen_srta @mcdonalds_br amiga, quando eu cr...
337  rt @albertosuzano: @slpng_giants_pt @youtubebr...
338  @bolsonarosp @mcdonalds_br @slpng_giants_pt @y...

In [11]:
#Neutro Separação da Lista
neut = train['Categorias'] == "Neutro"
neutro = train.loc[neut, ["Treinamento"]].copy()
neutro.head()

Treinamento
2   @sergiosepsep @mcdonalds_br @ifood eu já tive ...
8   @mcdonalds_br @jamesdeliverybr favor, voltem c...
17             queria era tá comendo um @mcdonalds_br
19  tá de sacanagem, né @mcdonalds_br ? https://t....
22  @baamarcal @mcdonalds_br @ifood pois é, essa e...

In [12]:
#Muito Irrelevante Separação da Lista
rel = train['Categorias'] == "Relevante"
relevante = train.loc[rel, ["Treinamento"]].copy()
relevante.head()

Treinamento
0   por isso eu prefiro o @mcdonalds_br https://t....
1   queria comer uma besteirinha, tipo um big tast...
13  como me explicar @mcdonalds_br pode existir mo...
14  @mcdonalds_br @fepaesleme domingo passado,  di...
23  @mcdonalds_br vocês cederam a pressão da sleep...

In [13]:
#Muito Irrelevante Separação da Lista
m_rel = train['Categorias'] == "Muito Relevante"
muito_relevante = train.loc[m_rel, ["Treinamento"]].copy()
muito_relevante.head()

Treinamento
4   fui no @mcdonalds_br do @rapososhopping e fiz ...
39  @slpng_giants_pt @mcdonalds_br mcdonald´s faça...
74  não aguento mais cara, toda vez que eu peço su...
75  @oiiuiz @mcdonalds_br eu já não comia lá msm, ...
76  pq uma água de 500ml está r$6,50  @mcdonalds_b...

In [14]:
#Limpeza dos Tweets
muito_irrelevante_cleaned = []

for row in muito_irrelevante.Treinamento:
    row = cleanup(row)
    muito_irrelevante_cleaned.append(row)
    
muito_irrelevante_cleaned

['@mvmarizinha @mcdonaldsbr mentira',
 'rt @thia6o0 @bolsonarosp @mcdonaldsbr @slpnggiantspt @youtubebrasil sim nunca percebeu as mensagens ocultas na logo ',
 'rt @rap24horasblog combo do travis scott no mcdonald’s faz enorme sucesso e esgota ingredientes do fast food  via @…',
 '@mvmarizinha @mcdonaldsbr sério kkkkkkkkkkkkkkkkkk',
 'olá @mcdonaldsbr o @luansantana está há procura de empresas pra apoiálo em uma live em prol do pantanal que tal abraçar essa ideia\n\nluan pelo pantanal',
 '@jotabarber @jpsantana09 mais conhecido como @mcdonaldsbr',
 'a felicidade que um @mcdonaldsbr proporciona hahahahaha novafotodoperfil ',
 '@lauras @mcdonaldsbr desejei',
 '@affonsoju @steviveiros @mcdonaldsbr e eu só queria um giga tasty mano',
 '@bolsonarosp @mcdonaldsbr @slpnggiantspt @youtubebrasil vai procurar um psiquiatra lunático kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk',
 '@rkteixeira7 @mcdonaldsbr ta achando q tu é o travor escotix seu fdp',
 '@nowornevergbye @claunutri4 @ograndepensado2 @jose

In [15]:
irrelevante_cleaned = []

for row in irrelevante.Treinamento:
    row = cleanup(row)
    irrelevante_cleaned.append(row)

In [16]:
neutro_cleaned = []

for row in neutro.Treinamento:
    row = cleanup(row)
    neutro_cleaned.append(row)

In [17]:
relevante_cleaned = []

for row in relevante.Treinamento:
    row = cleanup(row)
    relevante_cleaned.append(row)

In [18]:
muito_relevante_cleaned = []

for row in muito_relevante.Treinamento:
    row = cleanup(row)
    muito_relevante_cleaned.append(row)

In [19]:
#Transforma as palavras dentro de cada categoria em lista
muito_irrelevante_palavras = ' '.join(muito_irrelevante_cleaned).split()
irrelevante_palavras = ' '.join(irrelevante_cleaned).split()
neutro_palavras = ' '.join(neutro_cleaned).split()
relevante_palavras = ' '.join(relevante_cleaned).split()
muito_relevante_palavras = ' '.join(muito_relevante_cleaned).split()

#exemplo
neutro_palavras

['@sergiosepsep',
 '@mcdonaldsbr',
 '@ifood',
 'eu',
 'já',
 'tive',
 'problemas',
 'com',
 'o',
 'pedido',
 'do',
 'mc',
 'mas',
 'foi',
 'pelo',
 'ubereats',
 'liguei',
 'no',
 'ubereats',
 'fiz',
 'a',
 'reclamação',
 'e',
 'eles',
 'estornaram',
 'o',
 'valor',
 'no',
 'meu',
 'cartão',
 'tudo',
 'isso',
 'em',
 'menos',
 'de',
 '5',
 'min',
 '🙏🏻',
 'ninguém',
 'merece',
 'ficar',
 'puto',
 'com',
 'o',
 'pedido',
 'qdo',
 'chega',
 'e',
 'ainda',
 'levar',
 'canseira',
 'depois',
 'disso',
 '@mcdonaldsbr',
 '@jamesdeliverybr',
 'favor',
 'voltem',
 'com',
 'o',
 'refrigerante',
 'em',
 'lata',
 'que',
 'saco',
 'toda',
 'vez',
 'vem',
 'derramado',
 'metade',
 'do',
 'refrigerante',
 'e',
 'da',
 'ultima',
 'vez',
 'o',
 'refri',
 'ainda',
 'deu',
 'um',
 'banho',
 'na',
 'batata',
 'que',
 'saco',
 'queria',
 'era',
 'tá',
 'comendo',
 'um',
 '@mcdonaldsbr',
 'tá',
 'de',
 'sacanagem',
 'né',
 '@mcdonaldsbr',
 '@baamarcal',
 '@mcdonaldsbr',
 '@ifood',
 'pois',
 'é',
 'essa',
 'em

In [21]:
#Transformacao das categorias em Serie do Pandas e posteriormente remanejando em uma tabela relativa
muito_irrelevante_relativa = pd.Series(muito_irrelevante_palavras).value_counts(True)
irrelevante_relativa = pd.Series(irrelevante_palavras).value_counts(True)
neutro_relativa = pd.Series(neutro_palavras).value_counts(True)
relevante_relativa = pd.Series(relevante_palavras).value_counts(True)
muito_relevante_relativa = pd.Series(muito_relevante_palavras).value_counts(True)

#Demonstracao
neutro_relativa.head()

@mcdonaldsbr    0.053035
de              0.028874
o               0.026517
e               0.020625
que             0.020035
dtype: float64

In [ ]:
#String de Todas as palavras dentro da lista Train
total_palavras_train = (' '.join(muito_irrelevante_cleaned)) \
                    + (' '.join(irrelevante_cleaned)) \
                    + (' '.join(neutro_cleaned)) \
                    + (' '.join(relevante_cleaned)) \
                    + (' '.join(muito_relevante_cleaned))

#Relativa de todas as palavras
total_palavras_relativa = pd.Series(total_palavras_train.split()).value_counts(True)
print(total_palavras_relativa)

total_palavras = len(total_palavras_relativa)

print('')
print('A quantidade de palavras contidas nos tweets = {0}'.format(total_palavras))

In [ ]:
#verifica relevancia do Tweet, categorizando-o

def categoriza(tweet):
    #Tentativa da aplicacao de log
    '''tweet = tweet.cleanup()
    tweet_lista = tweet.split()
    log_tweet = []
    
    for relativa in tweet_lista:
        relativa = math.log10(relativa)
        log_tweet.append(relativa)'''
    
    #Multiplicacao das Probabilidadeas
    Prob_MIR_dado_Tweet = muito_irrelevante_relativa[tweet.split()].prod()
    Prob_IR_dado_Tweet = irrelevante_relativa[tweet.split()].prod()
    Prob_N_dado_Tweet = neutro_relativa[tweet.split()].prod()
    Prob_R_dado_Tweet = relevante_relativa[tweet_split()].prod()
    Prob_MR_dado_Tweet = muito_relevante_relativa[tweet_split()].prod()
    
    lista_probs = [Prob_MIR_dado_Tweet, Prob_IR_dado_Tweet, Prob_N_dado_Tweet, Prob_R_dado_Tweet, Prob_MR_dado_Tweet]
    
    #Comparacao de Probabilidades e Classificacao do Tweet
    categoria = 0
    
    for prob in lista_probs:
        if prob > categoria:
            categoria = prob
            
    if categoria == Prob_MIR_dado_Tweet:
        return 'Muito Irrelevante'
    
    if categoria == Prob_IR_dado_Tweet:
        return 'Irrelevante'
    
    if categoria == Prob_N_dado_Tweet:
        return 'Neutro'
    
    if categoria == Prob_R_dado_Tweet:
        return 'Relevante'
    
    else:
        return 'Muito Relevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**